# Assess Sensitivities
See which parameters in a mechanism have the largest impact on the quality of fit to experimental data.

In [1]:
from frhodo.api import FrhodoDriver
from typing import List
from scipy import stats as ss
from scipy.interpolate import interp1d
from pathlib import Path
from tqdm import tqdm
import pandas as pd
import numpy as np

Configuration

In [2]:
data_path = Path('sdl-examples/diacetyl/')

## Load in the Mechanism and Data
Create an instance of Frhodo from the driver and then load in the data for our diacetyl experiments

In [3]:
driver = FrhodoDriver.create_driver(headless=True)

This plugin does not support propagateSizeHints()
This plugin does not support propagateSizeHints()


Load in the data and ensure we set 

In [4]:
driver.load_files(
    data_path / 'experiment' / '2%-C4H6O2-in-98%-Kr-Oct-2016',
    data_path / 'mechanism', 
    data_path / 'simulation',
    aliases={
        'C4H6O2': 'CH3COCOCH3',
        'Krypton': 'Kr',
        '2,3-butanedione': 'CH3COCOCH3'
    }
)

Get initial simulation values

In [5]:
%%time
init_simulation = driver.run_simulations()

CPU times: user 3.37 s, sys: 30.1 ms, total: 3.4 s
Wall time: 3.4 s


Determine which ones are successful

In [6]:
was_success = [x.shape[0] > 10 for x in init_simulation]
print(f'{sum(was_success)} / {len(init_simulation)} shocks can be simulated.')

10 / 12 shocks can be simulated.


Load in the experimental data and the weights associated with each point

In [7]:
%%time
experiment_data, weights = driver.get_observables()

CPU times: user 1.69 s, sys: 29.5 ms, total: 1.72 s
Wall time: 1.72 s


## Measure performance of initial mechanism
Set up the loss function first then evaluate it on the current data. We will use the log-probability of observing the experimental data given the simulation results, following work by [Paulson et al.](https://www.sciencedirect.com/science/article/abs/pii/S0020722518314721).

In [8]:
def loss_function(simulation: np.ndarray, experiment: np.ndarray, weights: np.ndarray, error_bar: float = 1e-5) -> float:
    """Compaute the loss for a single experiment
    
    Args:
        simulated: Simulated observable
        experiment: Actual observable
        weights: Weights applied to each measurement in `experiment`
        error_bar: Estimated size of the uncertainty in the weight
    Returns:
        Log probability of observing the experimental data
    """
    # Truncate off portions that are weighted too low
    mask = weights > 1e-6
    experiment = experiment[mask, :]
    
    # Compute the simulated value at each point
    sim_func = interp1d(simulation[:, 0], simulation[:, 1], kind='cubic')
    predicted = sim_func(experiment[:, 0])
    
    # Get the difference
    difference = experiment[:, 1] - predicted
    
    # Compute the width of the uncertainty window
    std = error_bar / weights[mask]
    
    return ss.t(loc=0, scale=std, df=2.1).logpdf(difference).sum()

In [9]:
def total_loss(simulations: List[np.ndarray], 
               experiments: List[np.ndarray],
               weights: List[np.ndarray],
               was_success: List[np.ndarray],
               error_bar: float) -> float:
    """Compute the loss for all available experiments
    
    Args:
        simulations: List of simulated observables for differnet shock experiments
        experiments: List of measured observables
        weights: List of weights of each experiment
        was_success: Whether to use each 
    Returns:
        Log probability of the observed simulations"""
    
    log_probs = [
        loss_function(s, e, w, error_bar) 
        for s, e, w, m in 
        zip(simulations, experiments, weights, was_success)
        if m
    ]
    return np.sum(log_probs)

Print out a number that is worthless without context

In [10]:
init_loss = total_loss(
    init_simulation,
    experiment_data,
    weights,
    was_success,
    1e-4
)
init_loss

14032.630413734623

Ok! We can now quantify performance. Let's do something with that

## Test the sensitivity of different parameters
We'll vary a few parameters by 10% and see how much that affects the output

### Get the relevant parameters
Our Frhodo driver only supports Elementary, FallOff, and Three-body Reactions.

In [11]:
reaction_type = [
    type(x).__name__
    for x in  driver.window.mech.gas.reactions()
]
to_modify = [i for i, x in enumerate(reaction_type) if x in ['ElementaryReaction', 'FalloffReaction', 'ThreeBodyReaction']]
print(f'Modifying {len(to_modify)}/{len(reaction_type)} reactions')

Modifying 34/36 reactions


Get all parameters for these reactions. We need the index of the reaction, the index of the pressure being assessed (should be 0, as all of these assume 

In [12]:
parameters = driver.get_fittable_parameters(to_modify)
print(f'Found {len(parameters)} parameters to test')

Found 105 parameters to test


Loop over each parameter and evaluate the effect of changing it slightly

In [13]:
sensitivity = []
for parameter in tqdm(parameters):
    # Get the current value
    orig_value = driver.get_coefficients([parameter])[0]
    
    # Adjust it slightly and re-run simulations
    new_value = orig_value * 1.05
    driver.change_coefficient({parameter: new_value})
    new_simulations = driver.run_simulations()
    
    # Get the new loss value
    new_loss = total_loss(
        new_simulations,
        experiment_data,
        weights,
        was_success,
        1e-4
    )
    
    # Change it back
    driver.change_coefficient({parameter: orig_value})

    # Store the parameters and the change
    sensitivity.append(parameter + (new_loss - init_loss,))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 105/105 [02:59<00:00,  1.71s/it]


Compile into a dataframe

In [14]:
sensitivity = pd.DataFrame(sensitivity, columns=('reaction_id', 'pressure_id', 'coeff_name', 'sensitivity'))

Add in the reaction name and the magnitude of the sensitivity

In [15]:
sensitivity['reaction'] = sensitivity['reaction_id'].apply(lambda x: str(driver.window.mech.gas.reaction(x)))

In [16]:
sensitivity['sensitivity_mag'] = sensitivity['sensitivity'].abs()

Print out the most influential parameters

In [17]:
sensitivity.sort_values('sensitivity_mag', ascending=False).head()[['reaction', 'pressure_id', 'coeff_name', 'sensitivity']]

,reaction,pressure_id,coeff_name,sensitivity
89,CH3 + CH3COCOCH3 <=> CH2CO + CH3CO + CH4,0,temperature_exponent,-0.425496
0,C2H6 (+M) <=> 2 CH3 (+M),low_rate,activation_energy,0.408620
3,C2H6 (+M) <=> 2 CH3 (+M),high_rate,activation_energy,0.370785
5,C2H6 (+M) <=> 2 CH3 (+M),high_rate,temperature_exponent,0.313770
2,C2H6 (+M) <=> 2 CH3 (+M),low_rate,temperature_exponent,0.178253


Print out the least

In [18]:
sensitivity.sort_values('sensitivity_mag', ascending=False).tail()[['reaction', 'pressure_id', 'coeff_name', 'sensitivity']]

,reaction,pressure_id,coeff_name,sensitivity
68,CH2(S) + CH3 <=> C2H4 + H,0,temperature_exponent,0.0
71,CH2(S) + M <=> CH2(T) + M,0,temperature_exponent,0.0
74,C2H6 + CH2(S) <=> C2H5 + CH3,0,temperature_exponent,0.0
75,2 CH2(T) <=> C2H2 + H2,0,activation_energy,0.0
104,CH3 + CH3CO <=> C2H6 + CO,0,temperature_exponent,0.0
